In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

In [ ]:
def merge_peaks(df, chrom, start, end):
    """
    """
    return df[chrom].astype(str)+":"+df[start].astype(str)+"-"+df[end].astype(str)

In [ ]:
def get_integer_bins(vec):
    """excluding 0; 
    bins = [0, 1, ..., max(vec)] + 0.5
    mids = [   1, ..., max(vec)]
    """
    bins = np.arange(0, np.max(vec)+1) + 0.5
    mids = np.arange(1, np.max(vec)+1)
    frq_sig_a, _ = np.histogram(vec, bins=bins)
    
    return frq_sig_a, mids 

In [ ]:
f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac/all_AvsC_peaks_unique.bed'
df_allac_peaks = pd.read_csv(f, sep='\t', header=None)
df_allac_peaks['peak'] = merge_peaks(df_allac_peaks, 0,1,2)
df_allac_peaks

In [ ]:
%%time
# define cell order using RNA
f_rna = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/L23_allmultiome_proc_P6toP21.h5ad'
adata = sc.read(f_rna) # , backed='r')
cells_rna = np.array([c.split(' ')[0][:-len('-2023')] for c in adata.obs.index.values])
adata.obs.index = cells_rna
adata

In [ ]:
times = np.array([6,8,10,12,14,17,21]).astype(int)

In [ ]:
# same peak set 
# get ATAC
f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/P6_ATAC_L23_peakbycell_filtered.h5ad'
adata_pk = sc.read(f, backed='r')

# select and order by all peak set
peakset_idx = basicu.get_index_from_array(adata_pk.var['peak'].values, df_allac_peaks['peak'].values)
assert np.all(peakset_idx != -1)
peakset_idx.shape

In [ ]:
%%time

n_time = len(times)
n_type = 5
n_rep  = 2 
n_peak = len(peakset_idx)
tensor = np.zeros((n_time, n_type, n_rep, n_peak))

for i, ti in enumerate(times):
    print(ti)
    
    # select and order by cells (RNA)
    adatasub = adata[adata.obs['cond']==f'P{ti}']
    cells_rna = adatasub.obs.index.values
    cells_rna_type = pd.qcut(adatasub.obsm['pca_p17on'][...,0], n_type, labels=False)
    cells_rna_samp, categories = pd.factorize(adatasub.obs.Sample.values, )
    print(categories)
    
    # get ATAC
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/P{ti}_ATAC_L23_peakbycell_filtered.h5ad'
    adata_pk = sc.read(f)
    adata_pk = adata_pk[:,peakset_idx]
    
    adata_pk.obs = adata_pk.obs.set_index('cell')
    cells_atac = adata_pk.obs.index.values
    
    # per type
    for j in range(n_type):
        for k in range(n_rep):
            filter_cond = np.logical_and(cells_rna_type==j, cells_rna_samp==k) 
            cells_jk = np.intersect1d(cells_rna[filter_cond], cells_atac)
            print(i, j, k, len(cells_jk))
            tensor[i,j,k] = np.mean(adata_pk[cells_jk].X, axis=0)
        

In [ ]:
# save the results
tensor.shape

In [ ]:
output = "/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac/all_AvsC_peak_tensor.npy"
np.save(output, tensor)